In [1]:
import chess
import chess.engine
import numpy


# Creating the dataset

Now we need to convert the board representation to something meaningful.
A 3d matrix of sizes **8 x 8 x 14** where 8x8 repersents the board and the 14 represents the 7 different pieces 

In [2]:
# Build a simple dataset
squaresIndex = {
  'a': 0,
  'b': 1,
  'c': 2,
  'd': 3,
  'e': 4,
  'f': 5,
  'g': 6,
  'h': 7
}


# Example: h3 -> 17
def squareToIndex(square):
  letter = chess.square_name(square)
  return 8 - int(letter[1]), squaresIndex[letter[0]]


def splitDims(board):
  # This is the 3d matrix
  board3d = numpy.zeros((14, 8, 8), dtype=numpy.int8)

  # Here we add the pieces's view on the matrix
  for piece in chess.PIECE_TYPES:
    for square in board.pieces(piece, chess.WHITE):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece - 1][7 - idx[0]][idx[1]] = 1
    for square in board.pieces(piece, chess.BLACK):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece + 5][7 - idx[0]][idx[1]] = 1

  # Add attacks and valid moves too
  # So the network knows what is being attacked
  aux = board.turn
  board.turn = chess.WHITE
  for move in board.legal_moves:
      i, j = squareToIndex(move.to_square)
      board3d[12][i][j] = 1
  board.turn = chess.BLACK
  for move in board.legal_moves:
      i, j = squareToIndex(move.to_square)
      board3d[13][i][j] = 1
  board.turn = aux

  return board3d

Now, all we have to do is call **random_board()** to create random boards, **stockfish()** to get a score for how good each board is for white.

Then we convert each board to a 3d matrix using **splitDims()**, now creating the dataset is easy!

# TensorFlow!

In [3]:
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import tensorflow.keras.utils as utils
import tensorflow.keras.optimizers as optimizers


def buildModel(convSize, convDepth):
  board3d = layers.Input(shape=(14, 8, 8))

  # adding the convolutional layers
  x = board3d
  for _ in range(convDepth):
    x = layers.Conv2D(filters=convSize, kernel_size=3, padding='same', activation='relu')(x)
  x = layers.Flatten()(x)
  x = layers.Dense(64, 'relu')(x) # Rectified Linear Unit 
  x = layers.Dense(1, 'sigmoid')(x) # Sigmoid is key that neutral netword lear complex problem

  print(x)
  return models.Model(inputs=board3d, outputs=x)

In [4]:
model = buildModel(32, 4)

KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), name='dense_1/Sigmoid:0', description="created by layer 'dense_1'")


# It's training time!

In [5]:
import tensorflow.keras.callbacks as callbacks


def getDataset():
	container = numpy.load('dataset.npz')
	b, v = container['b'], container['v']
	v = numpy.asarray(v / abs(v).max() / 2 + 0.5, dtype=numpy.float32) # normalization (0 - 1)
	return b, v

x_train, y_train = getDataset()
x_train.transpose()

array([[[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 1, 1],
         [1, 0, 1, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 1, 1, 1],
         [1, 0, 1, ..., 0, 1, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 1],
         [1, 0, 0, ..., 0, 0, 1]],

        ...,

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 1],
         [0, 0, 1, ..., 0, 1, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
    

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint
model.compile(optimizer=optimizers.Adam(5e-4), loss='mean_squared_error')
model.summary()
checkpoint_filepath = '/tmp/checkpoint/'
model_checkpointing_callback = ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_best_only= True,
)
model.fit(x_train, y_train,
          batch_size=2048,
          epochs=1000,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4),model_checkpointing_callback])

model.save('model.h5')

# Algorithm between Minimax and neural network 

In [7]:
from tensorflow.keras import models
model = models.load_model('model.h5')

In [8]:
# Used for the minimax algorithm
def minimax_eval(board):
  board3d = splitDims(board)
  board3d = numpy.expand_dims(board3d, 0)

  return model(board3d)[0][0]

def minimax(board, depth, alpha, beta, maximizing_player):
  if depth == 0 or board.is_game_over():
    return minimax_eval(board)
  
  if maximizing_player:
    max_eval = -numpy.inf
    for move in board.legal_moves:
      board.push(move)
      eval = minimax(board, depth - 1, alpha, beta, False)
      board.pop()
      max_eval = max(max_eval, eval)
      alpha = max(alpha, eval)
      if beta <= alpha:
        break
    return max_eval
  else:
    min_eval = numpy.inf
    for move in board.legal_moves:
      board.push(move)
      eval = minimax(board, depth - 1, alpha, beta, True)
      board.pop()
      min_eval = min(min_eval, eval)
      beta = min(beta, eval)
      if beta <= alpha:
        break
    return min_eval


# this is the actual function that gets the move from the neural network
def get_ai_move(board, depth):
  max_move = None
  max_eval = -numpy.inf

  for move in board.legal_moves:
    board.push(move)
    eval = minimax(board, depth - 1, -numpy.inf, numpy.inf, False)
    board.pop()
    if eval > max_eval:
      max_eval = eval
      max_move = move
  
  return max_move

In [9]:
# Testing code AI (white) vs Minimax (black)
from main import * # From ass 2
neutralboard = chess.Board()
from IPython.display import clear_output

minimaxboard = board.Board.new()
counter = 0
while True:
    counter += 1
    print(counter)
    clear_output(wait=True)
    neutral = get_ai_move(neutralboard, 1)

    neutralmove = get_valid_user_move(minimaxboard, neutral)
    
    if neutralmove is False:
        print('Minimax win')
        break

    neutralboard.push(neutral)
    if neutralboard.is_game_over():
        print('Game over')
        break
    
    if (neutralmove == 0):
        if (minimaxboard.is_check(pieces.Piece.WHITE)):
            print("Minimax win")
            break
        else:
            print("Neutral win")
            break

    minimaxboard.perform_move(neutralmove)

    print("Neutral move: " + str(neutral))
    print(minimaxboard.to_string())

    minimaxmove = ai.AI.get_ai_move(minimaxboard, [])
    print("Minimax move: " + minimaxmove.to_string())

    if (minimaxboard.is_check(pieces.Piece.WHITE)):
        print("Minimax win")
        break
    
    if (minimaxmove == 0):
        if (minimaxboard.is_check(pieces.Piece.BLACK)):
            print("Neutral win")
            break
        else:
            print("Minimax win")
            break

    minimaxboard.perform_move(minimaxmove)
    print(minimaxboard.to_string())
    
    minimaxmove = minimaxmove.to_string()

    minimaxmove = chess.Move.from_uci(minimaxmove) #d2d4 -> Move(d2d4)
    neutralboard.push(minimaxmove)

    if neutralboard.is_game_over():
        print('game_over')
        break

Minimax win
